<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/three_plus_word_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove posts with only 1 or 2 words. Leave response length at 20.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "three_plus_words"
MODEL_NAME = "three_plus_words"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.5MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [4]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [5]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
len(dev_df)

2292907

In [7]:
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
train_df['split_response'] = train_df['response_text'].str.split()
train_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican,"[protecting, birth, is, not, the, same, as, protecting, life., you, may, very, well, pledge, to,..."
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican,"[you, need, to, protect, children, and, leave, my, body, to, me.]"
2,2,57265377,M,0,Sherry,thank you,Roger Williams,Congress_Republican,"[thank, you]"
3,3,57265377,M,0,Bob,thank you roger,Roger Williams,Congress_Republican,"[thank, you, roger]"
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican,"[unwanted, pregnancy, is, a, sad, and, unfortunate, situation, for, anyone, to, find, themselves..."


In [0]:
len(train_df)

9879016

In [8]:
# Remove the split responses that have length 1 or 2.
mask = (train_df['split_response'].str.len() > 2) 
train_df = train_df.loc[mask]
len(train_df)

8685575

In [9]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
mask2 = (dev_df['split_response'].str.len() > 2) 
dev_df = dev_df.loc[mask2]
len(dev_df)

2041945

In [0]:
len(dev_df)

390738

In [10]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3721104, 9)

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [13]:
train_df.op_gender.value_counts()

M    2725461
W     995643
Name: op_gender, dtype: int64

In [14]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do oversampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using oversampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [19]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 29 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [22]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   33.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
|
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [25]:
embedding_matrix.shape

(546265, 100)

In [26]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           54626500  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [30]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3721104 samples, validate on 2000000 samples
Epoch 1/30
3721104/3721104 [==============================] - 104s 28us/sample - loss: 0.5152 - acc: 0.7662 - val_loss: 0.4517 - val_acc: 0.8394
Epoch 2/30
3721104/3721104 [==============================] - 104s 28us/sample - loss: 0.4982 - acc: 0.7741 - val_loss: 0.4308 - val_acc: 0.8445
Epoch 3/30
3721104/3721104 [==============================] - 104s 28us/sample - loss: 0.4930 - acc: 0.7765 - val_loss: 0.4345 - val_acc: 0.8437
Epoch 4/30
3721104/3721104 [==============================] - 105s 28us/sample - loss: 0.4897 - acc: 0.7780 - val_loss: 0.4480 - val_acc: 0.8378
Epoch 5/30
3721104/3721104 [==============================] - 104s 28us/sample - loss: 0.4872 - acc: 0.7791 - val_loss: 0.4534 - val_acc: 0.8333
Epoch 6/30
3721104/3721104 [==============================] - 105s 28us/sample - loss: 0.4853 - acc: 0.7798 - val_loss: 0.4454 - val_acc: 0.8422
Epoch 7/30
3721104/3721104 [==============================] - 105s 28us/samp

In [31]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7870
Testing Accuracy:  0.8329


In [32]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6252946314091796


In [33]:
# can't get this to work
plot_model(history)

AttributeError: ignored

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [0]:
wrong_preds.op_gender.value_counts()

In [0]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

In [0]:
#random sample of mistakes
wrong_preds.head(20)

In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]